## Trabajo realizado por Benjamín Rubio y José Baboun
Importamos librerías necesarias

In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pylab
import networkx as nx
from collections import deque
import geopandas as gpd
import math
from shapely.geometry import Point
import itertools

## Funciones básicas del algoritmo Greedy

- get_size recibe como input el grafo (G), la lista de nodos ya visitados (V), la cantidad de nodos (N) y un nodo (u). A partir de esto se retorna el tamano de su ideal dentro de los nodos no visitados.
- visit toma como input grafo (G), la lista de nodos ya visitados (V) y un nodo (u). Con esto lo que hace es marcar como visitados todos los nodos dentro de su ideal de nodos no visitados. Retorna la lista actualizada y la cantidad de nodos visitados.

In [86]:
def get_size(G, V_, N, u):
    
    V = V_.copy()
    
    ans = 1;  V[u] = 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v]:
                ans += 1; V[v] = 1
                Q.append(v)
    
    return ans

def visit(G, V, u):
    V[u] = 1; ans = 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft(); ans += 1
        for v in G.predecessors(u):
            if not V[v]:
                V[v] = 1
                Q.append(v)
    
    return V, ans

- get_size_weight recibe como input el grafo (G), la lista de nodos ya visitados (V), la cantidad de nodos (N) y un nodo (u). A partir de esto se retorna el tamano de su ideal dentro de los nodos no visitados y su peso.

In [87]:
def get_size_weight(G, V_, N, u):
    V = V_.copy()
    
    s = 1; w = W[u];  V[u] = 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v]:
                s += 1; w += W[v]; V[v] = 1
                Q.append(v)
    
    return s, w

In [88]:
def get_size_weight_precalc(G, V_, S, N, u):
    
    if V_[u] == 1:
        return 0
    
    if S[u] <= 200:
        return 1
    
    V = V_.copy()
    
    s = 1; V[u] = 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v]:
                s += 1; V[v] = 1
                if S[v] > 200:
                    Q.append(v)
    
    return s

def greedyAppWeightReduce(G, V_, N, K, limit, wlim):
    
    V = V_.copy()
    
    E = []
    for u in range(N):
        if not V[u]:
            E.append((get_size(G, V, N, u), u))

    limit_set = set()
    for e in E:
        if e[0] > 0 and e[0] <= limit:
            limit_set.add(e[1])

    ans = []
    count = 0
    while count < K:
        
        S = [0 for u in range(N)]
        for v in limit_set:
            _, S[v] = get_size_weight(G, V, N, v)
        
        maxv = 0; u = -1
        for v in limit_set:
            s = get_size_weight_precalc(G, V, S, N, v)
            if s > maxv and S[v] <= wlim:
                u = v
                maxv = s
        if u == -1:
            break

        V, _ = visit(G, V, u)
        ans.append(u)
        limit_set.remove(u)
        count += 1

    total = len(G.nodes)
    v = sum([1 for node in G.nodes if V[node] == 1])
    return ans, v/total

In [89]:
def add_visit(G, N, V, u):
    
    V_ = [0 for u in range(N)]
    
    V[u] += 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V_[v]:
                V[v] += 1
                V_[v] = 1
                Q.append(v)
                
def deactivate(G, V, u):
    V[u] = 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v]:
                V[v] = 1
                Q.append(v)

In [90]:
def get_ideal(G, V_, u):
    V = V_.copy()
    s = [u];  V[u] = 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v]:
                s.append(v)
                V[v] = 1; Q.append(v)
    
    return s

def get_ideal_(G, N, u, _id):
    V = [0 for u in range(N)]
    #w = G.nodes[u]['weight']; s = list();  V[u] = 1
    w = W[u]; s = list();  V[u] = 1
    Q = deque([]); Q.append(u)
    while Q:
        u = Q.popleft()
        for v in G.predecessors(u):
            if not V[v]:
                #w += G.nodes[v]['weight']; s.append(back[v])
                w += W[v]; s.append(_id[v])
                V[v] = 1; Q.append(v)
    
    return s

In [91]:
def particionar(G, V, N, node):
    V = [0 if i in get_ideal(G, V, node) else 1 for i in range(N)]

    R = len(get_ideal(G, V, node))        
    low = 0; high = len(get_ideal(G, V, node))
    while low != high:
        mid = (low + high) // 2
        P, sP = greedyAppWeightReduce(G, V, N, 1, mid, 100000000)
        if R - sP <= mid:
            high = mid
        else:
            low = mid + 1
    P, sP = greedyAppWeightReduce(G, V, N, 1, low, 100000000)
    return P

def partition_ideal(G, V, N, node):
    V = [0 if i in get_ideal(G, V, node) else 1 for i in range(N)]

    R = len(get_ideal(G, V, node))        
    low = 0; high = len(get_ideal(G, V, node))
    while low != high:
        mid = (low + high) // 2
        P, sP = greedyAppWeightReduce(G, V, N, 1, mid, 100000000)
        if R - sP <= mid:
            high = mid
        else:
            low = mid + 1
    P, sP = greedyAppWeightReduce(G, V, N, 1, low, 100000000)
    ideal = get_ideal(G, V, P[0])
    return P, ideal